In [ ]:
import pandas as pd
file=pd.read_parquet(r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\evaluation_data\input\raw_data.parquet')
print(file.head())
print('hi')

In [21]:
import pandas as pd
file=pd.read_parquet(r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\evaluation_data\input\raw_data.parquet' ,engine='pyarrow')

file['timestamp']= pd.to_datetime(file['timestamp'])

df=file

#df["timestamp"].diff()
a=(df.groupby('unit'))
print(a.head())



       latitude  longitude                 timestamp  unit
0      40.23426  -77.13796 2023-12-05 23:13:40+00:00  1000
1      40.23422  -77.13796 2023-12-05 23:13:40+00:00  1000
2      40.23419  -77.13795 2023-12-05 23:13:40+00:00  1000
3      40.23412  -77.13792 2023-12-05 23:13:40+00:00  1000
4      40.23412  -77.13792 2023-12-05 23:13:40+00:00  1000
46500  31.13662  -81.58010 2023-12-11 21:58:42+00:00  2000
46501  31.13678  -81.58042 2023-12-11 21:58:44+00:00  2000
46502  31.13708  -81.58125 2023-12-11 21:58:49+00:00  2000
46503  31.13716  -81.58146 2023-12-11 21:58:50+00:00  2000
46504  31.13716  -81.58146 2023-12-11 21:58:50+00:00  2000


In [ ]:
#Process1

import pandas as pd
from datetime import datetime, timedelta

def process_gps_data(parquet_file):
    # Read Parquet file into a DataFrame
    df = pd.read_parquet(parquet_file,engine='pyarrow')
    df['timestamp']= pd.to_datetime(df['timestamp'])

    # Sort the DataFrame by unit and timestamp
    df.sort_values(by=['unit', 'timestamp'], inplace=True)

    # Create a new column 'trip_number' to identify trips
    df['trip_number'] = (df.groupby('unit')['timestamp']
                         .diff()
                         .gt(pd.Timedelta(hours=7))
                         .cumsum()
                         .fillna(0)
                         .astype(int))

    # Iterate over unique units and write trip-specific CSV files
    for unit, unit_df in df.groupby('unit'):
        for trip_number, trip_df in unit_df.groupby('trip_number'):
            # Construct CSV file name
            csv_filename = f'{unit}_{trip_number}.csv'

            # Write trip-specific CSV file
            trip_df[['latitude', 'longitude', 'timestamp']].to_csv(csv_filename, index=False)

if __name__ == "__main__":
    # Replace 'your_parquet_file.parquet' with the actual file path
    process_gps_data(r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\evaluation_data\input\raw_data.parquet' )


In [ ]:
#Process1

import pandas as pd
from datetime import datetime, timedelta

def process_gps_data(parquet_file):
    # Read Parquet file into a DataFrame
    df = pd.read_parquet(parquet_file,engine='pyarrow')
    df['timestamp']= pd.to_datetime(df['timestamp'])

    # Sort the DataFrame by unit and timestamp
    df.sort_values(by=['unit', 'timestamp'], inplace=True)

    # Create a new column 'trip_number' to identify trips
    df['trip_number'] = (df.groupby('unit')['timestamp']
                         .diff()
                         .gt(pd.Timedelta(hours=7))
                         .cumsum()
                         .fillna(0)
                         .astype(int))

    # Iterate over unique units and write trip-specific CSV files
    for unit, unit_df in df.groupby('unit'):
        for trip_number, trip_df in unit_df.groupby('trip_number'):
            # Construct CSV file name
            csv_filename = f'{unit}_{trip_number}.csv'

            # Write trip-specific CSV file
            trip_df[['latitude', 'longitude', 'timestamp']].to_csv(csv_filename, index=False)

if __name__ == "__main__":
    # Replace 'your_parquet_file.parquet' with the actual file path
    process_gps_data(r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\evaluation_data\input\raw_data.parquet' )


In [ ]:
#Process 2

import os
import requests
import argparse
import concurrent.futures
import load_dotenv
# Load environment variables from .env file
load_dotenv.load_dotenv()
print(os.getenv("TOLLGURU_API_KEY"))

def upload_to_tollguru(csv_file, output_dir):
    # API endpoint and parameters
    url = f'{os.getenv("TOLLGURU_API_URL")}/gps-tracks-csv-upload'
    params = {'mapProvider': 'osrm', 'vehicleType': '5AxlesTruck'}

    # Prepare headers with API key
    headers = {'x-api-key': os.getenv("TOLLGURU_API_KEY"), 'Content-Type': 'text/csv'}

    # Read CSV file
    with open(csv_file, 'rb') as file:
        # Send request to TollGuru API
        response = requests.post(url, params=params, data=file, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Construct JSON file name
            json_filename = os.path.join(output_dir, os.path.splitext(os.path.basename(csv_file))[0] + '.json')

            # Save JSON response to file
            with open(json_filename, 'w') as json_file:
                json_file.write(response.text)

            print(f'Successfully processed {csv_file} and saved JSON response to {json_filename}')
        else:
            print(f'Error processing {csv_file}. Status code: {response.status_code}, Message: {response.text}')

def process_gps_files(to_process, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List all CSV files in the input directory
    csv_files = [os.path.join(to_process, file) for file in os.listdir(to_process) if file.endswith('.csv')]

    # Use concurrent futures to upload files concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(upload_to_tollguru, csv_files, [output_dir] * len(csv_files))

if __name__ == "__main__":
    # Set up command-line argument parser
    parser = argparse.ArgumentParser(description='Upload GPS tracks to TollGuru API')
    parser.add_argument(r'/output/process1')
    parser.add_argument(r'/output')
    args = parser.parse_args()

    # Upload GPS tracks to TollGuru API
    process_gps_files(r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\sample_data\output\process1', r'C:\Users\admin\Desktop\MapUp-Data-Assessment-E\sample_data\output\process2')
    print('done')

usage: ipykernel_launcher.py [-h] /output/process1 /output
ipykernel_launcher.py: error: the following arguments are required: /output/process1, /output


SystemExit: 2

C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
